In [ ]:
from icecream import ic
from rp import *

channel = JupyterDisplayChannel()
channel.display()

mask_channel = JupyterDisplayChannel()
mask_channel.display()


textured_model_channel = JupyterDisplayChannel()
textured_model_channel.display()


text_channel = JupyterDisplayChannel()
text_channel.display()


In [ ]:
import noise_warping as nw
import torch
from icecream import ic

if "device" not in vars():
    device = select_torch_device(prefer_used=True)

# uvl_image= rp.load_image('/root/CleanCode/Projects/deepfloyd_init_test/blender/BlenderOutputKevinSpinner/UV_Label_Exr/Image0155.exr',use_cache=True)
# uvl_image = rp.resize_image_to_fit(uvl_image,256,256,interp='nearest')
# texture_image = rp.load_image('https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png',use_cache=True)


if "texture_image" not in vars():
    tic()
    # texture_image=get_checkerboard_image(height=512*3,width=512*3)
    # texture_image = rp.as_float_image(rp.as_rgb_image(texture_image))
    # texture_image[:]=-1
    # texture_image=rp.as_torch_image(texture_image)
    # texture_image=torch.randn_like(texture_image)
    texture_image = torch.randn(4, 20000, 20000)
    # noise_backdrop = torch.randn(4, 40000, 40000)  # Not too big
    texture_image = texture_image.to(device)
    # noise_backdrop = noise_backdrop.to(device)
    # texture_image=texture_image.to(device)
    ptoc()

# texture_image= load_image('/Users/ryan/Downloads/KevinSpinner/frame_0002_point-cloud_geo_geo-retexture.jpg')


if not 'background_torch_randn' in vars():
    @memoized
    def background_torch_randn(dims):
        return torch.randn(dims).to(device)





# if __name__=="__main__":
def warped_noise_from_uvl_path(path):
    uvl_image = rp.load_image(path, use_cache=True)
    F=3;uvl_image = cv_resize_image(uvl_image, (240 * F + 1, 135 * F + 1), "nearest")
    # F=6;uvl_image = cv_resize_image(uvl_image, (114 * F + 1, 64 * F + 1), "nearest")

    uvl_image[:,:,0] *= uvl_image[:,:,2] == 0 #Where blue, destroy UV so we don't repeat anything...
    uvl_image[:,:,1] *= uvl_image[:,:,2] == 0 #Where blue, destroy UV so we don't repeat anything...

    uvl_image = rp.as_torch_image(uvl_image).to(device)

    out = nw.uv_mapping_discretized(uv_image = uvl_image, tex_image=texture_image, w=20, debug_plot=False)

    out.noisewarp = out.ryan_filter*out.area**.5

    std_mea = out.noisewarp.flatten()
    std_mea = std_mea[std_mea!=0]

    #True where we should include the static background noise
    background_mask_matrix = out.area[0] == 0
    foreground_mask_matrix = 1-background_mask_matrix.float()

    rp.display_image(
        rp.horizontally_concatenated_images(
            rp.as_numpy_image(out.noisewarp / 10 + 0.5),
            rp.as_numpy_array(background_mask_matrix),
        )
    )
    ic(
        out.noisewarp.min(),
        out.noisewarp.max(),
        out.noisewarp.mean(),
        out.noisewarp.std(),
        std_mea.std(),
    )


    mask_channel.update(rp.as_rgb_image(rp.as_numpy_array(foreground_mask_matrix)))

    # noisewarp_with_noise_background = background_noise * background_mask_matrix[None] + out.noisewarp

    # Variance-preserving downsample
    lowres_noisewarp_sum = torch.nn.functional.avg_pool2d(out.noisewarp[None], kernel_size=F, stride=F)[0]
    lowres_noisewarp_sum = lowres_noisewarp_sum * F * F #Turn the avg into a sum

    # Be nice to the cracks and seams...
    lowres_foreground_area_matrix = torch.nn.functional.avg_pool2d(foreground_mask_matrix[None,None], kernel_size=F, stride=F)[0,0]
    lowres_foreground_area_matrix = lowres_foreground_area_matrix * F**2 #Adjust variance

    lowres_noisewarp = lowres_noisewarp_sum / lowres_foreground_area_matrix[None] ** .5
    lowres_noisewarp = lowres_noisewarp.nan_to_num()

    background_noise = background_torch_randn(lowres_noisewarp.shape)

    lowres_noisewarp = lowres_noisewarp + (lowres_foreground_area_matrix[None] == 0) * background_noise

    
    #TODO: Scale down a large texture-mapped noise and then bin them and weight it properly...only after that should you add the background noise
    #That way we can get as much consistency as possible, fill in as much of those pesky seams as we can...

    output_noise = rp.as_numpy_image(lowres_noisewarp)
    text_channel.grid_update([[x] for x in [
        " ".join(map(str,("STD MEA STD ",std_mea.std(),))),
        # " ".join(map(str,("OUTSTD ",noisewarp_with_noise_background.std(),))),
        # " ".join(map(str,("OUTMEAN ",noisewarp_with_noise_background.mean(),))),
        " ".join(map(str,("OUTNOISE_STD ",output_noise.std(),))),
        " ".join(map(str,("OUTNOISE_STD ",output_noise.shape,))),
    ]])

    



    return output_noise


# ans = process_uvl_path('/root/CleanCode/Projects/deepfloyd_init_test/blender/BlenderOutputKevinSpinner/UV_Label_Exr/Image0155.exr')
# display_image(ans/10+.5)

# warped_noise_image / 10 + .5
from IPython.display import clear_output

output_folder = make_directory(
    get_unique_copy_path("noisewarp_outputs/KevinSpinnerAlignedFloored")
)

ic.disable()
input_paths = sorted(
    rp_glob(
        "/root/CleanCode/Projects/deepfloyd_init_test/blender/BlenderOutputKevinSpinner/UV_Label_Exr/*.exr"
    )
)
input_paths = sorted(
    rp_glob(
        "/root/CleanCode/Projects/deepfloyd_init_test/blender/ProperSpinnerJul1/BlenderOutputKevinSpinner/UV_Label_Exr/*.exr"
    )
)[::-1]

for index,input_path in enumerate(input_paths):
    warped_noise_image = warped_noise_from_uvl_path(input_path)
    clear_output()
    output_path = path_join(output_folder, ('%05i'%index)+get_file_name(input_path))
    print("IN:", input_path)
    print("OUT:", output_path)
    # display_image(warped_noise_image)

    saveable_image = warped_noise_image #/ 10 + 0.5
    channel.update(rp.as_rgb_image(saveable_image))
    # channel.update(rp.as_rgb_image(saveable_image))
    run_as_new_thread(
        save_image, saveable_image, output_path
    )  # Quantization doesn't matter too much...

In [ ]:
saveable_image.std()

At http://ryan-burgert-controlnet-tests.workbench.prod.netflix.net:54545
In /root/CleanCode/Projects/deepfloyd_init_test/noise_warping_python/noisewarp_outputs/KevinSpinner_copy16/*.exr
Use this code to view noise:

"""
import rp
import glob

def glob_search(query: str, replacements: dict) -> list:
    """
    Query is like "/path/to/{x:05}/image_*/{y}.png"
    Replacements is like {"x":5,"y":100}
    Returns a list of globbed paths
    """
    query = query.format(**replacements)
    paths = glob.glob(query)
    paths = sorted(paths)
    return paths

@rp.memoized
def load_image_bytes(path: str) -> bytes:
    image = rp.load_image(path, use_cache=True)
    # image = rp.resize_image_to_fit(image, height = 512,interp='area')
    image = rp.rotate_image(image, 90)
    title = '\n'.join(path.split('/')[-3:])
    image = rp.labeled_image(image, title, size=60)
    image_bytes = rp.encode_image_to_bytes(image, 'png', quality=95)
    return image_bytes

    # return rp.file_to_bytes(path) #Just loads the file raw
"""

In [ ]:
import AlgoNotes as nw
import torch
from icecream import ic

if "device" not in vars():
    device = select_torch_device(prefer_used=True)

if "kevin_texture_image" not in vars():
    kevin_texture_image = "/root/CleanCode/Projects/deepfloyd_init_test/blender/frame_0002_point-cloud_geo_geo-retexture.1001.jpg"
    kevin_texture_image = as_torch_image(
        as_float_image(as_rgb_image(load_image(kevin_texture_image)))
    ).to(device)


def textured_model_from_uvl_path(path):
    uvl_image = rp.load_image(path, use_cache=True)

    uvl_image = as_torch_image(uvl_image).to(device)

    output = nw.uv_mapping_discretized(
        uv_image=uvl_image, tex_image=kevin_texture_image, w=1, device=device
    )
    ic(texture_image.shape, output.ryan_filter.shape)

    out_image = output.linear

    is_blue = uvl_image[2, 1:, 1:][None] > 0
    out_image = torch.maximum(is_blue, out_image)

    out_image = as_numpy_image(out_image)
    return out_image


from IPython.display import clear_output

output_folder = make_directory(
    get_unique_copy_path("noisewarp_outputs/KevinSpinnerAlignedTextured")
)

ic.disable()
input_paths = sorted(
    rp_glob(
        "/root/CleanCode/Projects/deepfloyd_init_test/blender/BlenderOutputKevinSpinner/UV_Label_Exr/*.exr"
    )
)
input_paths = sorted(
    rp_glob(
        "/root/CleanCode/Projects/deepfloyd_init_test/blender/ProperSpinnerJul1/BlenderOutputKevinSpinner/UV_Label_Exr/*.exr"
    )
)[::-1]
for input_path in input_paths:
    textured_model_image = textured_model_from_uvl_path(input_path)
    clear_output()
    output_path = path_join(output_folder, get_file_name(input_path))
    print("IN:", input_path)
    print("OUT:", output_path)
    # display_image(warped_noise_image)

    saveable_image = textured_model_image
    textured_model_channel.update(saveable_image)
    run_as_new_thread(
        save_image, saveable_image, output_path
    )  # Quantization doesn't matter too much...

In [ ]:
kevin_texture_image = as_torch_image(
    as_float_image(as_rgb_image(load_image(kevin_texture_image)))
).to(device)

In [ ]:
del kevin_texture_image